In [1]:
#import cell
import QuantLib as ql
from IPython.core.display import display, Markdown
from instruments.interestRateInstrument.swaption import Swaption
from jupyterUtils import export
from marketdata import init_marketdata
from marketdata.init_marketdata import today
from instruments.interestRateInstrument.irs import IRS
from marketdata.interestRateIndices import InterestRateIndex
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import SwapDirection, TradeDirection
asdf =1

To highlight the relationship between the 6M forward curve and the par rate $P_i$ of forward starting swap we can set up the following two swaps depicted below. Since the $EndDate-StartDate$ for these two swaps is six months their par rate or fair price can directly be read from the 6M Forward curve.

In [2]:
print('StartDate: ' + str(today+ql.Period(30,ql.Months)))
swap1 = IRS(notional=100,
            timeToSwapStart=ql.Period(30,ql.Months),
            timeToSwapEnd=ql.Period(36,ql.Months),
            swapDirection=SwapDirection.PAYER,
            index=InterestRateIndex.EURIBOR6M,
            fixed_rate=0.01
           )
print('ParRate: %.6f%%' % (swap1.get_par_rate()*100))

StartDate: November 10th, 2021
ParRate: -0.002346%


In [3]:
print('StartDate: ' + str(today+ql.Period(31,ql.Months)))
swap2 = IRS(notional=100,
            timeToSwapStart=ql.Period(31,ql.Months),
            timeToSwapEnd=ql.Period(37, ql.Months),
            swapDirection=SwapDirection.PAYER,
            index=InterestRateIndex.EURIBOR6M,
            fixed_rate=0.01
            )
print('ParRate: %.6f%%' % (swap2.get_par_rate()*100))

StartDate: December 10th, 2021
ParRate: 0.011976%


In [4]:
swap3 = IRS(notional=100,
            timeToSwapStart=ql.Period(30, ql.Months),
            timeToSwapEnd=ql.Period(36, ql.Months),
            swapDirection=SwapDirection.PAYER,
            index=InterestRateIndex.EURIBOR6M,
            fixed_rate=-0.01)

In [5]:
#calculatin SA_CCR delta for swaptions that convert into these swaps

swaption1 = Swaption(underlyingSwap=swap1,
                     optionMaturity=ql.Period(30, ql.Months),
                     tradeDirection=TradeDirection.LONG)

SA_CCR.calculate_sa_ccr_delta(swaption1)

0.004138768807253323

In [6]:
swaption2 = Swaption(underlyingSwap=swap2,
                     optionMaturity=ql.Period(31, ql.Months),
                     tradeDirection=TradeDirection.LONG)

SA_CCR.calculate_sa_ccr_delta(swaption2)

0.005109036476851662

The following is a very deep in the money swaption with negative strike and price. The delta should be close to 1. It is however close to zero without offsetting.

In [7]:
swaption3 = Swaption(underlyingSwap=swap3,
                     optionMaturity=ql.Period(30, ql.Months),
                     tradeDirection=TradeDirection.LONG)

SA_CCR.calculate_sa_ccr_delta(swaption3)

0.9996934007461944

In [8]:
print(swaption1.get_delta())
print(swaption2.get_delta())


4.875082678775719
5.248490507272288


In [ ]:
export('issue_regulatory_swaption_delta_2.ipynb')